# Infer MITOCHONDRIA - part 5

--------------

## OBJECTIVE:  Infer sub-cellular component  MITOCHONDRIA  in order to understand interactome 



Dependencies:
The MITOCHONDRIA  inference rely on the CYTOSOL, which is SOMA&~NUCLEI.  



## IMPORTS

In [ ]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage import morphology
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.utils.file_io import read_input_image, list_image_files, export_ome_tiff
from infer_subc.utils.img import *

%load_ext autoreload
%autoreload 2

from infer_subc.organelles.mitochondria import infer_MITOCHONDRIA

# IMAGE PROCESSING Objective 5:  infer MITOCHONDRIA
> Back to  [OUTLINE: Objective #5](#summary-of-objectives)

## summary of steps (Workflow #1 & #2)

INPUT
- channel  3
- CY mask

PRE-PROCESSING
-  smoothe / remove noise

CORE-PROCESSING
-  segment objects

- POST-PROCESSING
  - filter objects

OUTPUT
- object MITOCHONDRIA 



------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [ ]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[5]

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

In [ ]:
chan_name = 'nuclei'
out_path = data_root_path / "inferred_objects" 
object_name = 'CY_object'

CY_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
CY_object = CY_bioim.image


# WORKFLOW #1 

Generally following the Allen Cell Segmenter procedure, but doing more aggressive contrast scaling than their prescribed contrast scaling.

> Using Allen Cell Segmenter Tom20as a potential Mitochondrian segmenter to start with  start from [Allen Cell](https://www.allencell.org/cell-observations/category/mitochondria).   using [seg_tomm20.py]("../../../../aics-segmentation/aicssegmentation/structure_wrapper/seg_tomm20.py")

## summary of steps

INPUT
- channel  2
- CY mask

PRE-PROCESSING
-  smoothe / remove noise

CORE-PROCESSING
-  'vesselness' enhancement
-  threshold objects

- POST-PROCESSING
  - filter objects

OUTPUT
- object MITOCHONDRIA 


In [ ]:
##########################################################################
# DEFAULT PARAMETERS:
#   note that these parameters are supposed to be fixed for the structure
#   and work well accross different datasets
# default_params = defaultdict(str)

default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 10
})


################################
# calculate a filter dimension for median filtering which considers the difference in scale of Z
z_factor = scale[0]//scale[1]
med_filter_size = 4 #2D 
med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter
default_params['z_factor'] = z_factor
default_params['scale'] = scale


### INPUT

In [ ]:

###################
# INPUT
###################
struct_img_raw = img_data[2,:,:,:].copy()

# DEFAULT PARAMETERS:
intensity_norm_param = [3.5, 15] # from Allen Cell Segmenter LAMP1  workflow
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

med_filter_size =3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img_raw) #  [0., 23]


### PRE-PROCESSING

In [ ]:
###################
# PRE_PROCESSING
###################

intensity_norm_param = [0, 9] # from Allen Cell Segmenter LAMP1  workflow

# Linear-ish smoothing
raw_mito = intensity_normalization( struct_img_raw ,  scaling_param=intensity_norm_param)

med_filter_size =3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

struct_img = median_filter(raw_mito,    size=med_filter_size  )

structure_img_smooth = image_smoothing_gaussian_3d(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


# log_img, d = log_transform( structure_img_smooth ) 
# struct_img = intensity_normalization(  log_img  ,  scaling_param=[0] )  

struct_img = structure_img_smooth

### CORE PROCESSING

In [ ]:
###################
# CORE_PROCESSING
###################
# dot and filiment enhancement - 2D


################################
## PARAMETERS for this step ##
vesselness_sigma = [1.5]
vesselness_cutoff = 0.16
# 2d vesselness slice by slice
response = vesselnessSliceBySlice(structure_img_smooth, sigmas=vesselness_sigma, tau=1, whiteonblack=True)
bw = response > vesselness_cutoff


# # from Sec61b workflows - playground references Tom20
# f2_param = [[1.5, 0.16]]
# bw = filament_2d_wrapper(structure_img_smooth, f2_param)

### POST-PROCESSING

In [ ]:

# # 3D
###################
# POST_PROCESSING
###################

################################
## PARAMETERS for this step ##
min_area = 10
################################

MT_object = remove_small_objects(bw > 0, min_size=min_area, connectivity=1, in_place=False)
cleaned_img = size_filter(bw, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= min_area**2, 
                                                         method = "3D" ,
                                                         connectivity=1)


#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:

if viewer is None:
    viewer = napari.view_image(
        cleaned_img,
        scale=scale
    )
else: 
    viewer.add_image(
        cleaned_img,
        scale=scale
    )

viewer.scale_bar.visible = True
viewer.add_image(
    structure_img_smooth,
    scale=scale
)

viewer.add_image(
    MT_object,
    scale=scale
)

### DEFINE `infer_MITOCHONDRIA` function

In [ ]:
##########################
#  infer_MITOCHONDRIA
##########################
def _infer_MITOCHONDRIA(struct_img, CY_object,  in_params) -> tuple:
    """
    Procedure to infer MITOCHONDRIA  from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the MITOCHONDRIA signal

    CY_object: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of MITOCHONDRIA
        parameters: dict
            updated parameters in case any needed were missing
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    scaling_param =  [0,9]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

   # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 3   
    structure_img = median_filter(struct_img,    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.3
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_3d(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    # log_img, d = log_transform( struct_img ) 
    # struct_img = intensity_normalization( log_img ,  scaling_param=[0] )  
    # struct_img = intensity_normalization( struct_img ,  scaling_param=[0] )  


   ###################
    # CORE_PROCESSING
    ###################
    ################################
    ## PARAMETERS for this step ##
    vesselness_sigma = [1.5]
    vesselness_cutoff = 0.16
    # 2d vesselness slice by slice
    response = vesselnessSliceBySlice(structure_img_smooth, sigmas=vesselness_sigma, tau=1, whiteonblack=True)
    bw = response > vesselness_cutoff

    out_p["vesselness_sigma"] = vesselness_sigma 
    out_p["vesselness_cutoff"] = vesselness_cutoff 


    ###################
    # POST_PROCESSING
    ###################

    # MT_object = remove_small_objects(bw > 0, min_size=small_object_max**2, connectivity=1, in_place=False)
    small_object_max = 10
    struct_obj = size_filter(bw, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "3D", #"slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    retval = (struct_obj,  out_p)
    return retval


# WORKFLOW #2 (WIP)
as per 6/22 CellProfiler pipeline from MCZ
 > ADAPTIVE... not sure exactly how to impliment this...

  >ReduceNoise performs non-local means noise reduction. Instead of only using a neighborhood of pixels around a central pixel for denoising, such as in GaussianFilter, multiple neighborhoods are pooled together. The neighborhood pool is determined by scanning the image for regions similar to the area around the central pixel using a correlation metric and a cutoff value.
   
## summary of steps (Workflow #1 & #2)

INPUT
- ch 3
- CY mask

PRE-PROCESSING
-  non-local noise reduction
  - size:4, distance:2, cut-off:0.1

CORE-PROCESSING
  - adaptive Otsu
    - diameter: (2,200)
  - two classes
    - threshold smoothing scale: 0
    - threshold correction factor: .1
    - threshold bounds: (0.14 1)
    - adaptive window: 20 pixels

POST-PROCESSING
  - N/A





### INPUT

In [ ]:

###################
# INPUT
###################
struct_img_raw = img_data[2,:,:,:].copy()

# DEFAULT PARAMETERS:
intensity_norm_param = [3.5, 15] # from Allen Cell Segmenter LAMP1  workflow
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

med_filter_size =2  

gaussian_smoothing_sigma = 10
gaussian_smoothing_truncate_range = 3.0

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img_raw) #  [0., 23]


### PRE-PROCESSING

In [ ]:
###################
# PRE_PROCESSING
###################

intensity_norm_param = [0] # 

# Linear-ish smoothing
raw_mito = intensity_normalization( struct_img_raw ,  scaling_param=intensity_norm_param)

med_filter_size =3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

struct_img = median_filter(raw_mito,    size=med_filter_size  )

structure_img_smooth = image_smoothing_gaussian_3d(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


# log_img, d = log_transform( structure_img_smooth ) 
# struct_img = intensity_normalization(  log_img  ,  scaling_param=[0] )  

struct_img = structure_img_smooth

### CORE PROCESSING

In [ ]:
###################
# CORE_PROCESSING
###################
def _enhance_speckles(image, radius, volumetric=True):
    radius = radius / 2
    if volumetric:
        selem = ball(radius)
    else:
        selem = disk(radius)     

    # if radius >10:
    #         minimum = scipy.ndimage.minimum_filter(image, footprint=selem)
    #         maximum = scipy.ndimage.maximum_filter(minimum, footprint=selem)
    #         result = data - maximum
    # else:
    result =  white_tophat(image)

    return result

def enhance_neurites(image, radius, volumetric = True):
    if volumetric:
        selem = ball(radius)
    else:
        selem = disk(radius)     
    white = white_tophat(image,selem)
    black = black_tophat(image, selem)
    result = image + white - black
    result[result > 1] = 1
    result[result < 0] = 0

    return result

    



In [ ]:
# enhance spreckles - 40
big_struct_rad = 40
big_img = _enhance_speckles(struct_img.copy(),big_struct_rad, True)



In [ ]:

# enhance spreckles - 10
sm_struct_rad = 20
sm_img = _enhance_speckles(struct_img.copy(),sm_struct_rad, True)


#adaptive_otsu(big_struct) # three class - middle foreground
# adaptive window- 20
#size: 10,100
# threshold smooth 1.34
# threshold correction 1
# threshold (0.1497,1)
#fill holes
# "adatpive" thresholds are NOT currently working... will use      
# "Masked Object Thresholding" - 3D
low_level_min_size = 10
bw_big, _bw_low_level = MO(big_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
 # or this?                                               
#    struct_obj = struct_img > filters.threshold_otsu(struct_img)
#     threshold_value_log = threshold_otsu_log(struct_img)

#     threshold_factor = 0.9 #from cellProfiler
#     thresh_min = .1
#     thresh_max = 1.
#     threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
#     struct_obj = struct_img > threshold


# enhance speckles
#   adaptive_sauvola(sm_struct) 
# adaptive window- 10
#size: 2,10
# threshold smooth 1.34
# threshold correction 1
# threshold (0.05,1)
#fill holes
# "adatpive" thresholds are NOT currently working... will use      
# "Masked Object Thresholding" - 3D
low_level_min_size = 2
bw_sm, _bw_low_level = MO(sm_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
 # or this?                                               
#    struct_obj = struct_img > filters.threshold_otsu(struct_img)
#     threshold_value_log = threshold_otsu_log(struct_img)
#     threshold_factor = 0.9 #from cellProfiler
#     thresh_min = .1
#     thresh_max = 1.
#     threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
#     struct_obj = struct_img > threshold



### POST-PROCESSING

In [ ]:
###################
# POST_PROCESSING
###################


# 3D
# cleaned_img = remove_small_objects(removed_holes>0, 
#                                                             min_size=minArea, 
#                                                             connectivity=1, 
#                                                             in_place=False)
small_object_max = 10
cleaned_img_big = size_filter(bw_big, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**2,
                                                         method = "slice_by_slice",
                                                         connectivity=1)


#                                                             in_place=False)
small_object_max = 2
cleaned_img_sm = size_filter(bw_sm, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**2,
                                                         method = "slice_by_slice",
                                                         connectivity=1)



cleaned_img = np.logical_or(cleaned_img_big, cleaned_img_sm)



#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:

if viewer is None:
    viewer = napari.view_image(
        cleaned_img,
        scale=scale
    )
else: 
    viewer.add_image(
        cleaned_img,
        scale=scale
    )

viewer.scale_bar.visible = True
viewer.add_image(
    cleaned_img_big,
    scale=scale
)

viewer.add_image(
    big_img,
    scale=scale
)
viewer.add_image(
    cleaned_img_big,
    scale=scale
)
viewer.add_image(
    sm_img,
    scale=scale
)

# DEFINE `infer_MITOCHONDRIA_CP` function (WIP)

In [ ]:
# copy this to base.py for easy import
# mangle so we can call from base.py
def _infer_MITOCHONDRIA_CP(struct_img: np.ndarray,  CY_object:np.array,  in_params:dict) -> tuple:
    """
    Procedure to infer MITOCHONDRIA from linearly unmixed input as per CellProfiler procedure

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the MITOCHONDRIA signal (should be CY masked)
    CY_object: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
    
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param


    # 2D smoothing
    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 9   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 3.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    #    edges = filters.scharr(struct_img)
    # struct_img, d = log_transform( struct_img ) 
    # struct_img = intensity_normalization(  struct_img,  scaling_param=[0] )
    ###################
    # CORE_PROCESSING
    ###################
    # "Masked Object Thresholding" - 3D
    local_adjust = 0.25
    low_level_min_size = 100
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)
    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 

    ###################
    # POST_PROCESSING
    ###################
    # 3D cleaning

    hole_max = 80  
    # discount z direction
    struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_max = 35
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "3D", #"slice_by_slice" 
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    labels_out = watershed(
                                                image=np.abs(ndi.sobel(struct_img)),
                                                markers=NU_labels,
                                                connectivity=np.ones((3, 3, 3), bool),
                                                mask= np.logical_or(struct_obj, NU_labels > 0),
                                                )

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    all_labels = np.unique(labels_out)[1:]

    total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    # now use all the NU labels which AREN't keep_label and add to mask and re-label
    masked_composite_soma = struct_img.copy()
    new_NU_mask = np.logical_and( NU_labels !=0 ,NU_labels != keep_label)

    # "Masked Object Thresholding" - 3D
    masked_composite_soma[new_NU_mask] = 0
    struct_obj, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)
    # 3D cleaning
    struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "3D", #"slice_by_slice" 
                                                            connectivity=1)


    retval = (struct_obj, out_p)
    return retval

# TEST  `infer_MITOCHONDRIA` function

In [ ]:
chan_name = 'nuclei'
out_path = data_root_path / "inferred_objects" 
object_name = 'CY_object'

CY_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
CY_object = CY_bioim.image

###################
# INPUT
###################
struct_img_raw = img_data[2,:,:,:].copy()


# DEFAULT PARAMETERS:
intensity_norm_param = [3.5, 15] # from Allen Cell Segmenter LAMP1  workflow
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

med_filter_size =3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img_raw) #  [0., 23]

intensity_norm_param = [0, 9] # from Allen Cell Segmenter LAMP1  workflow
# Linear-ish smoothing
raw_mito = intensity_normalization( struct_img_raw ,  scaling_param=intensity_norm_param)


MT_object, out_p =  infer_MITOCHONDRIA(raw_mito.copy(), CY_object, default_params) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper
# possibly need to do some post-post-processing to make suer that there is only a single SO_Object?


chan_name = 'nuclei'
out_path = data_root_path / "inferred_objects" 
object_name = 'MT_object'

MT_object_filen = export_ome_tiff(MT_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)


In [ ]:

viewer2.add_image(
    raw_mito,
    scale=scale 
)

viewer2.add_image(
    MT_object,
    scale=scale
)
viewer2.scale_bar.visible = True

